In [2]:
import requests
import pprint
import pandas as pd
import yfinance as yf
from datetime import datetime, timedelta
import pandas_datareader.data as web

In [3]:
AV_apikey = "X5KOQR5HRJ1NMVD9"
ticker = "AAPL"

In [6]:
url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={AV_apikey}'
r = requests.get(url)
data = r.json()
pprint.pprint(data)

{'annualReports': [{'capitalExpenditures': '10959000000',
                    'cashflowFromFinancing': '-108488000000',
                    'cashflowFromInvestment': '3705000000',
                    'changeInCashAndCashEquivalents': '5760000000',
                    'changeInExchangeRate': 'None',
                    'changeInInventory': '1618000000',
                    'changeInOperatingAssets': '7719000000',
                    'changeInOperatingLiabilities': '1142000000',
                    'changeInReceivables': '417000000',
                    'depreciationDepletionAndAmortization': '11519000000',
                    'dividendPayout': '15025000000',
                    'dividendPayoutCommonStock': '15025000000',
                    'dividendPayoutPreferredStock': 'None',
                    'fiscalDateEnding': '2023-09-30',
                    'netIncome': '96995000000',
                    'operatingCashflow': '110543000000',
                    'paymentsForOperatingActivities

In [7]:
df = pd.DataFrame({"Year": [],
                   "FreeCashFlow": []})

In [8]:
for dict in data["annualReports"]:
    year = dict["fiscalDateEnding"][:4]
    ocf = int(dict["operatingCashflow"])
    capex = int(dict["capitalExpenditures"])
    fcf = ocf - capex
    entry = [year, fcf]
    df.loc[len(df.index)] = entry

In [9]:
df

,Year,FreeCashFlow
0,2023,99584000000
1,2022,111443000000
2,2021,92953000000
3,2020,73365000000
4,2019,58896000000
5,2018,64121000000
6,2017,51430000000
7,2016,52683000000
8,2015,69778000000
9,2014,99562000000


In [10]:
risk_free_rate = yf.Ticker("^TNX").info.get("previousClose") / 100
sp500 = yf.download("^GSPC", 
                    start = "1996-11-22",#(datetime.now() - timedelta(days = 10000)).strftime("%Y-%m-%d"), 
                    end = datetime.now().strftime("%Y-%m-%d"))
market_return = ((1 + sp500["Adj Close"].pct_change().dropna().mean()) ** 252) - 1


def find_SGR(ticker):
    try:
        stock = yf.Ticker(ticker)
        ROE = stock.info.get("returnOnEquity")
        dividends_paid = abs(stock.cashflow.loc["Cash Dividends Paid"].head(1).values[0])
        net_income = stock.incomestmt.loc["Net Income"].head(1).values[0]
        retention_rate = 1 - (dividends_paid / net_income)
        
        return ROE * retention_rate
    
    except:
        return 100 # arbitrary maximum

# Long-term growth rate (nominal interest rate)
def find_LTGR():
    data = web.DataReader('DGS10', 'fred', datetime(2023, 1, 1), datetime.now())
    return data.iloc[-1]["DGS10"] / 100

def find_WACC(ticker):
    stock = yf.Ticker(ticker)
    beta = stock.info.get("beta")
    cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

    interest_expense = stock.financials.loc["Interest Expense"].head(1).values[0]
    total_debt = stock.balancesheet.loc["Total Debt"].head(1).values[0]
    cost_of_debt = interest_expense / total_debt

    market_cap = stock.info.get("marketCap")
    equity_weight = market_cap / (market_cap + total_debt)
    debt_weight = 1 - equity_weight
    tax_rate = stock.financials.loc["Tax Rate For Calcs"].head(1).values[0]

    wacc = (cost_of_equity * equity_weight) + (cost_of_debt * debt_weight * (1 - tax_rate))
    """print("Risk-Free Rate:", risk_free_rate)
    print("Market Return:", market_return)
    print("Equity Market Risk Premium:", (market_return - risk_free_rate))
    print("Beta:", beta)
    print("Cost of Equity:", cost_of_equity)
    print("Equity Weight:", equity_weight)
    print("Cost of Debt:", cost_of_debt)
    print("Debt Weight:", debt_weight)
    print("Tax Rate:", tax_rate)"""
    return wacc

[*********************100%%**********************]  1 of 1 completed


In [17]:
def DCF(ticker, years_to_project = 10):
    url = f'https://www.alphavantage.co/query?function=CASH_FLOW&symbol={ticker}&apikey={AV_apikey}'
    r = requests.get(url)
    data = r.json()
    # pprint.pprint(data)
    
    fcf = pd.DataFrame({"Year": [],
                       "FreeCashFlow": []})
    
    for dict in data["annualReports"]:
        year = int(dict["fiscalDateEnding"][:4])
        ocf = int(dict["operatingCashflow"])
        capex = int(dict["capitalExpenditures"])
        cashflow = ocf - capex
        entry = [year, cashflow]
        fcf.loc[len(fcf.index)] = entry

    fcf = fcf.sort_values(by = "Year")
    
    stock = yf.Ticker(ticker)
    # fcf = stock.get_cash_flow().loc["FreeCashFlow", :].dropna()
    # past_years = [fcf.index[i].year for i in range(len(fcf))]
    # fcf = fcf.set_axis(pd.Index(past_years)).sort_index().reset_index().rename(columns = {"index":"Year"})
    
    avg_growth = fcf["FreeCashFlow"].pct_change().mean()

    latest_historical_info = fcf.iloc[-1]
    future_years = [i for i in range(latest_historical_info["Year"] + 1, latest_historical_info["Year"] + years_to_project + 1)]

    for yr in future_years:
        proj_fcf = (latest_historical_info["FreeCashFlow"] * (1 + avg_growth) ** (yr - latest_historical_info["Year"]))
        fcf.loc[len(fcf.index)] = [yr, proj_fcf]

    display(fcf)
    
    perpetual_growth_rate = min(find_SGR(ticker), find_LTGR())
    discount_rate = find_WACC(ticker)

    terminal_value = fcf.iloc[-1]["FreeCashFlow"] * (1 + perpetual_growth_rate) / (discount_rate - perpetual_growth_rate)
    future_fcf_sum = fcf.tail(years_to_project)["FreeCashFlow"].sum() + terminal_value

    balance_sheet = stock.quarterly_balance_sheet
    cash_casheq = balance_sheet.loc["Cash Cash Equivalents And Short Term Investments"].iloc[0]
    total_debt = balance_sheet.loc["Total Debt"].iloc[0]
    equity_value = future_fcf_sum + cash_casheq - total_debt
    shares_outstanding = balance_sheet.loc["Share Issued"].iloc[0]
    
    dcf_pps = (equity_value / shares_outstanding)
    return round(dcf_pps, 2)

In [18]:
def compare(ticker, years_to_project = 10):
    comp_stock = yf.Ticker(ticker)
    curr_price = comp_stock.info.get("previousClose")
    DCF_price = DCF(ticker, years_to_project)
    comp_name = comp_stock.info.get("shortName")
    return comp_name + " is currently at $" + str(curr_price) + ", and its " + str(years_to_project) + "-year DCF projection is $" + str(DCF_price)


In [21]:
compare("AAL", 5)

,Year,FreeCashFlow
13,2010.0,-7.210000e+08
12,2011.0,-8.670000e+08
11,2012.0,-6.030000e+08
10,2013.0,-2.439000e+09
9,2014.0,-2.231000e+09
8,2015.0,9.800000e+07
7,2016.0,7.930000e+08
6,2017.0,-1.227000e+09
5,2018.0,-2.120000e+08
4,2019.0,-4.530000e+08


'American Airlines Group, Inc. is currently at $11.28, and its 5-year DCF projection is $14912.04'

In [23]:
lst = [0, 1]
a, b = lst
print(b)

1
